# Getting The Data
Brian Bahmanyar

___

** NOTE:** This notebook will not run without implementing your own get_token function to return a Quandl API token
* just use the data from /data

[Quandl](https://www.quandl.com) provides free daily financial data which will be used in the analyses to come. They also provide a free, but somewhat lackluster Python API.

In [11]:
import numpy as np
import pandas as pd
import Quandl
from my_token import get_token

> get_token is a function that returns my Quandl API token, replace with your information as necessary. Or just use the data below, provided in /data.

Below is a function written to serve as a wrapper around Quandl's Python API and provide some needed functionality.

In [8]:
def get_adj_close(token, tickers, start, end="", ratios=[], log_transforms=[]):
    """
    Args:
        tickers (list): collection of ticker symbols for which to collect adj. close 
                daily prices for
        start (string, format: 2013-01-01): start date for which to collect prices after
        end (string, format: 2013-01-01): optional end date, today if not specified
        ratios (list): collection of tuples of tickers from 'tickers' list to calculate 
                price ratios for (the stock with larger mean is numerator)
        log_transforms (list): collection of tickers from 'tickers' to include additional 
                natural log transformed copies
    
    Returns (dataframe): all adj. close prices, ratios, and log transforms specified
    """ 
    result = {}
    
    for ticker in tickers:
        try:
            result[ticker] = Quandl.get('WIKI/'+ticker, trim_start=start, trim_end=end, authtoken=token)['Adj. Close']
        except DatasetNotFound:
            print('ERROR:')
            print(ticker, 'is not a vaild ticker')

    for ratio in ratios:
        try:
            ticker1, ticker2 = ratio
            if result[ticker1].mean() > result[ticker2].mean():
                result[ticker1+'/'+ticker2] = result[ticker1]/result[ticker2]
            else:
                result[ticker2+'/'+ticker1] = result[ticker2]/result[ticker1]
        except KeyError:
            print('ERROR:')
            print(ticker1, 'or', ticker2, 'are not in the list of specified tickers')
    
    for log_transform in log_transforms:
        try:
            result['ln('+log_transform+')'] = np.log(result[log_transform])
        except KeyError:
            print('ERROR:')
            print(log_transform, 'is not in the list of specified tickers')
    
    return pd.DataFrame(result).dropna() # drop na here because of differences in lenght of history for stocks

> A copy of this function is placed into api_wrapper.py for use in other notebooks.

***

### Now to Get the Data We Need

In [9]:
tech_bundle = get_adj_close(get_token(),
                            ['FB', 'AMZN', 'AAPL'], 
                            start='2013-01-01', 
                            ratios=[('FB','AMZN'), ('FB','AAPL'), ('AMZN','AAPL')],
                            log_transforms=['FB', 'AMZN', 'AAPL'])

In [10]:
pairs_bundle = get_adj_close(get_token(),
                            ['VZ', 'T', 'KO', 'PEP', 'XOM', 'CVX'], 
                            start='1990-01-01', 
                            ratios=[('VZ','T'), ('KO','PEP'), ('XOM','CVX')])

In [12]:
tech_bundle.head()

,AAPL,AAPL/FB,AMZN,AMZN/AAPL,AMZN/FB,FB,ln(AAPL),ln(AMZN),ln(FB)
Date,,,,,,,,,
2013-01-02,73.295822,2.617708,257.31,3.510568,9.189643,28.00,4.294504,5.550282,3.332205
2013-01-03,72.370116,2.606054,258.48,3.571640,9.307886,27.77,4.281793,5.554818,3.323956
2013-01-04,70.354805,2.446273,259.15,3.683473,9.010779,28.76,4.253551,5.557407,3.358986
2013-01-07,69.940953,2.377327,268.46,3.838381,9.125085,29.42,4.247651,5.592702,3.381675
2013-01-08,70.129189,2.413255,266.38,3.798418,9.166552,29.06,4.250339,5.584924,3.369363


In [13]:
pairs_bundle.head()

,CVX,CVX/XOM,KO,PEP,PEP/KO,T,VZ,VZ/T,XOM
Date,,,,,,,,,
1990-01-02,6.724925,0.864271,2.791292,6.310486,2.260776,6.506971,12.543602,1.927718,7.781034
1990-01-03,6.622892,0.859756,2.753444,6.249591,2.269736,6.393413,12.516094,1.957655,7.703223
1990-01-04,6.530134,0.856365,2.743982,6.187714,2.255012,6.272066,12.061367,1.923029,7.625413
1990-01-05,6.437377,0.848530,2.715596,6.102264,2.247118,5.969798,11.785666,1.974215,7.586508
1990-01-08,6.502307,0.844102,2.772368,6.212268,2.240780,6.070554,11.882713,1.957435,7.703223


In [4]:
# tech_bundle.to_csv('data/tech_bundle.csv')
# pairs_bundle.to_csv('data/pairs_bundle.csv')